<a href="https://colab.research.google.com/github/SYKIM5396/TIL/blob/main/pandas_grouping%26unstack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/SYKIM5396/New_Game_Design/main/vgames2.csv')
df.head()

,Unnamed: 0,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
0,1,Candace Kane's Candy Factory,DS,2008.0,Action,Destineer,0.04,0,0,0
1,2,The Munchables,Wii,2009.0,Action,Namco Bandai Games,0.17,0,0,0.01
2,3,Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010.0,Adventure,Alchemist,0,0,0.02,0
3,4,Deal or No Deal: Special Edition,DS,2010.0,Misc,Zoo Games,0.04,0,0,0
4,5,Ben 10 Ultimate Alien: Cosmic Destruction,PS3,2010.0,Platform,D3Publisher,0.12,0.09,0,0.04


In [3]:
# K는 *0.001로 M은 그냥 지우기 그후 각 요소에 pd.eval 함수 적용. 그후 타입변경
df['NA_Sales'] = df['NA_Sales'].replace({'K': '*0.001','M':'' }, regex=True).apply(pd.eval).astype(float)
df['EU_Sales'] = df['EU_Sales'].replace({'K': '*0.001','M':'' }, regex=True).apply(pd.eval).astype(float)
df['JP_Sales'] = df['NA_Sales'].replace({'K': '*0.001','M':'' }, regex=True).apply(pd.eval).astype(float)
df['NA_Sales'] = df['NA_Sales'].replace({'K': '*0.001','M':'' }, regex=True).apply(pd.eval).astype(float)

In [4]:
#Year값에 4자리 년도가 아닌 숫자들이 많다. 70~90년대는 1900더해주고, 나머지 20보다 작은 숫자들은 2000을 더해서 4자리 년도로 바꿔준다.
for i in range(df.shape[0]):
    if df["Year"][i] < 20  :
            df.loc[i,["Year"]] += 2000
    if df["Year"][i] > 70 and df["Year"][i] < 100 :
            df.loc[i,["Year"]] += 1900

In [5]:
#Total_sales 칼럼 추가
df["Total_sales"] = df.loc[:,["NA_Sales",	"EU_Sales",	"JP_Sales",	"Other_Sales"]].sum(axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  


In [6]:
#Genre에 따른 Total_sales 평균 분산 최대 최소값 구하기
#.groupby().agg() 메서드 연습
df[['Genre','Total_sales']].groupby('Genre').agg(['mean','var','max','min'])

Total_sales                        
                    mean        var     max  min
Genre                                           
Action          0.687377   2.195465   24.57  0.0
Adventure       0.215133   0.469353   14.36  0.0
Fighting        0.646541   1.554989   16.11  0.0
Misc            0.593114   3.065410   37.26  0.0
Platform        1.238111  12.201491   61.74  0.0
Puzzle          0.514585   5.242089   48.66  0.0
Racing          0.765471   4.070932   44.58  0.0
Role-Playing    0.554727   3.075079   31.43  0.0
Shooter         1.129121   7.859286   54.49  0.0
Simulation      0.554069   1.908704   29.14  0.0
Sports          0.742213   7.726960  112.00  0.0
Strategy        0.268706   0.547463    7.60  0.0

In [7]:
#Year와 Genre에 따른 Total_sales의 평균을 계층적 indexing 없이 구하라
df.groupby(['Year','Genre']).Total_sales.mean().unstack()

Genre,Action,Adventure,Fighting,Misc,Platform,Puzzle,Racing,Role-Playing,Shooter,Simulation,Sports,Strategy
Year,,,,,,,,,,,,
1980.0,0.660000,NaN,1.480000,1.302500,NaN,NaN,NaN,NaN,6.775000,NaN,NaN,NaN
1981.0,1.141200,NaN,NaN,NaN,4.436667,2.155000,0.930000,NaN,1.930000,0.860000,0.380000,NaN
1982.0,0.695556,NaN,NaN,1.670000,1.932000,6.426667,1.515000,NaN,1.462000,NaN,1.010000,NaN
1983.0,0.787143,0.780000,NaN,0.020000,1.206000,1.490000,NaN,NaN,0.930000,NaN,1.560000,NaN
1984.0,1.790000,NaN,NaN,0.000000,0.000000,0.520000,1.696667,NaN,18.810000,NaN,1.895000,NaN
1985.0,1.830000,NaN,0.000000,NaN,15.907500,0.712500,NaN,NaN,1.440000,0.030000,0.590000,NaN
1986.0,2.450000,NaN,NaN,NaN,0.386667,NaN,2.630000,0.980000,0.510000,NaN,1.723333,NaN
1987.0,1.070000,4.880000,3.125000,NaN,0.970000,NaN,NaN,0.313333,0.675000,NaN,0.207500,NaN
1988.0,1.230000,NaN,NaN,NaN,9.417500,6.630000,3.320000,0.066667,0.970000,0.020000,1.530000,NaN


In [8]:
#Action 장르를 가지는 데이터들 중 Platform 그룹별로 Total_sales의 평균, 분산, 최대, 최소값을 구하라
df[df.Genre=='Action'].groupby(['Platform']).Total_sales.agg(['mean','var','max','min'])

,mean,var,max,min
Platform,,,,
2600,0.924918,0.497212,4.24,0.27
3DS,0.283022,0.490388,5.33,0.00
DC,0.350000,0.367500,1.05,0.00
DS,0.466356,1.010172,11.57,0.00
GB,1.708333,4.660177,5.38,0.00
GBA,0.532695,0.516757,4.23,0.00
GC,0.593960,0.863318,6.19,0.02
GEN,1.423333,6.077633,4.27,0.00
N64,1.192162,3.546234,10.09,0.02


In [9]:
#데이터중 Genre 값에 따른 Platform 컬럼의 숫자를 구하고 Genre를 기준으로 각 값의 비율을 구하여라
df1 = df[['Genre','Platform']].groupby(['Genre','Platform']).size().unstack().fillna(0)
df1.loc[:,:] = (df1.values /df1.sum(axis=1).values.reshape(-1,1))
df1.T

Genre,Action,Adventure,Fighting,Misc,Platform,Puzzle,Racing,Role-Playing,Shooter,Simulation,Sports,Strategy
Platform,,,,,,,,,,,,
2600,0.018457,0.001563,0.002361,0.002884,0.010181,0.019031,0.004827,0.000000,0.018349,0.001156,0.004699,0.000000
3DO,0.000000,0.000781,0.000000,0.000000,0.000000,0.001730,0.000000,0.000000,0.000000,0.001156,0.000000,0.000000
3DS,0.055068,0.028906,0.016529,0.030565,0.031674,0.034602,0.008850,0.057316,0.005352,0.034682,0.011106,0.022059
DC,0.000908,0.008594,0.014168,0.000000,0.002262,0.000000,0.004827,0.002697,0.002294,0.001156,0.004272,0.000000
DS,0.103782,0.186719,0.042503,0.226067,0.104072,0.410035,0.053902,0.134862,0.032110,0.328324,0.063221,0.116176
GB,0.001815,0.003906,0.000000,0.004614,0.021493,0.025952,0.001609,0.014160,0.000765,0.005780,0.003845,0.010294
GBA,0.050530,0.028906,0.027155,0.063437,0.160633,0.070934,0.051488,0.048550,0.030581,0.020809,0.037591,0.026471
GC,0.030560,0.015625,0.049587,0.020761,0.082579,0.020761,0.050684,0.018206,0.036697,0.013873,0.046988,0.016176
GEN,0.000908,0.001563,0.005903,0.000577,0.007919,0.000000,0.000805,0.002023,0.000765,0.000000,0.001282,0.001471
